Active Fleet inventory data extract cleaning process

Set up daily download from ePlan emailed each weekday as .xls, connect to PowerQuery

In [ ]:
let

In [ ]:
Source = Excel.Workbook(File.Contents("C:\Users\vwarren1\OneDrive - CBRE, Inc\Attachments\Fleet Inventory.xlsx"), null, true),

Filter Rows, Expand Data, Promote Headers

In [ ]:
#"Filtered Rows" = Table.SelectRows(Source, each ([Name] = "Details")),
    #"Expanded Data" = Table.ExpandTableColumn(#"Filtered Rows", "Data", {"Column1", "Column2", "Column3", "Column4", "Column5", "Column6", "Column7", "Column8", "Column9", "Column10", "Column11", "Column12", "Column13", "Column14", "Column15", "Column16", "Column17", "Column18", "Column19", "Column20"}, {"Data.Column1", "Data.Column2", "Data.Column3", "Data.Column4", "Data.Column5", "Data.Column6", "Data.Column7", "Data.Column8", "Data.Column9", "Data.Column10", "Data.Column11", "Data.Column12", "Data.Column13", "Data.Column14", "Data.Column15", "Data.Column16", "Data.Column17", "Data.Column18", "Data.Column19", "Data.Column20"}),
    #"Promoted Headers" = Table.PromoteHeaders(#"Expanded Data", [PromoteAllScalars=true]),

Changed data types for the following fields to text in order to Join with other data sources: Client Account Code, Year, AcctMethod Code, Client Unit Number, Make, Base Model, Vehicle Type, VIN, Fuel Type, Driver County, Driver City, Driver State, Driver Zip, Owner, Vehicle Telematics Serial #, Vehicle Contractor #, Vehicle GPS Supplier, Vehicle GPS Model#

In [ ]:
#"Changed Type" = Table.TransformColumnTypes(#"Promoted Headers",{{"Client Account Code", type text}, {"Year", type text}, {"AcctMethod Code", type text}, {"Client Unit Number", type text}, {"ePlan Unit Number", Int64.Type}, {"Make", type text}, {"Base Model", type text}, {"Vehicle Type", type text}, {"VIN", type text}, {"Fuel Type", type text}, {"Months In Service", Int64.Type}, {"Driver County", type text}, {"Driver City", type text}, {"Driver State", type text}, {"Driver Zip", type text}, {"Owner", type text}, {"Vehicle Telematics Serial #", type text}, {"Vehicle Contractor #", type text}, {"Vehicle GPS Supplier", type text}, {"Vehicle GPS Model#", type text}}),

Remove records unneeded for data analysis, including MDHD Trucks, Equipment, Golf Carts, Latin American and Canadian Vehicles, and null records

In [ ]:
#"Filtered out NonUS" = Table.SelectRows(#"Changed Type", each ([Owner] <> "LEASEPLAN CANADA") and ([Driver State] <> "PR") and ([Driver State] <> "AB")and ([Driver State] <> "BC") and ([Driver State] <> "MB") and ([Driver State] <> "NB") and ([Driver State] <> "NL") and ([Driver State] <> "NT") and ([Driver State] <> "NS") and ([Driver State] <> "NU") and ([Driver State] <> "ON") and ([Driver State] <> "PE") and ([Driver State] <> "QC") and ([Driver State] <> "SK") and ([Driver State] <> "YT")
),
#"Filtered out MDHR etc" = Table.SelectRows(#"Filtered out NonUS", each ([Driver State] <> null) and ([Make] <> "CITR" and [Make] <> "CLUB CAR" and [Make] <> "EQUIPMENT" and [Make] <> "EZ GO" and [Make] <> "GEM WAEV" and [Make] <> "HINO" and [Make] <> "HORTON VANS" and [Make] <> "INTERNATIONAL" and [Make] <> "ISUZU" and [Make] <> "JOHN DEERE" and [Make] <> "KENWORTH" and [Make] <> "MAXUS" and [Make] <> "POLARIS" and [Make] <> "TRIPLE-L" and [Make] <> "C6" and [Base Model] <> "GMC ME6500 C7D064" and [Base Model] <> "GMT-400" and [Base Model] <> "LCF") and [Base Model] <> "F-650 DIESEL" and [Base Model] <> "F-750 GAS" ),
    #"Filtered Out Null Rows" = Table.SelectRows(#"Filtered out MDHR etc", each ([Client Account Code] <> null)),

Replaced fuel type values for accurate joining

In [ ]:
= Table.FuzzyNestedJoin(#"Replaced Value4", {"Base Model"}, Model_VehType, {"Find Model"}, "Model_VehType", JoinKind.LeftOuter, [IgnoreCase=true, IgnoreSpace=true])


Append joined columns to existing table

In [ ]:
= Table.ExpandTableColumn(#"Merged Queries", "Model_VehType", {"Find Model", "Replace Model", "Vehicle Type Corrected"}, {"Find Model", "Replace Model", "Vehicle Type Corrected"})
= Table.SelectRows(#"Expanded Model_VehType1", each true)

Remove and reorder original/dirty columns from FleetInventory

In [ ]:
= Table.RemoveColumns(#"Filtered Rows3",{"Base Model", "Find Model"})
= Table.RenameColumns(#"Removed Columns1",{{"Replace Model", "BaseModel"}})
= Table.ReorderColumns(#"Renamed Columns",{"Client Account Code", "AcctMethod Code", "Client Unit Number", "ePlan Unit Number", "Year", "Make", "BaseModel", "Vehicle Type", "VIN", "Fuel Type", "Months In Service", "Driver County", "Driver City", "Driver State", "Driver Zip", "Owner", "Vehicle Telematics Serial #", "Vehicle Contractor #", "Vehicle GPS Supplier", "Vehicle GPS Model#", "Vehicle Type Corrected"})
= Table.RemoveColumns(#"Reordered Columns",{"Vehicle Type"})
= Table.ReorderColumns(#"Removed Columns2",{"Client Account Code", "AcctMethod Code", "Client Unit Number", "ePlan Unit Number", "Year", "Make", "BaseModel", "Vehicle Type Corrected", "VIN", "Fuel Type", "Months In Service", "Driver County", "Driver City", "Driver State", "Driver Zip", "Owner", "Vehicle Telematics Serial #", "Vehicle Contractor #", "Vehicle GPS Supplier", "Vehicle GPS Model#"})
= Table.RenameColumns(#"Reordered Columns1",{{"Vehicle Type Corrected", "Vehicle Type"}})

Remove null and duplicate fields as a result of Join


In [ ]:
= Table.SelectRows(#"Renamed Columns1", each ([BaseModel] <> null))